In [1]:
import geopandas as gpd
import utils.analysis as an

c:\Users\Sergio\anaconda3\envs\ox_GIZ_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Zonas Medellin

In [ ]:
zo = ['aguacatala_HL', 'moravia_LH', 'floresta_HH']
z = zo[0]
for z in zo:
    print(f"Procesando zona: {z}")

    zone_edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
    zone_nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    land_use = gpd.read_file('../data/input/shape/land_use/land_use_medellin.shp')

    zone_edges = zone_edges.to_crs('epsg:32618')
    land_use = land_use.to_crs('epsg:32618')
    zone_edges = zone_edges.set_index('osmid')


    # Definition of categories
    categories = ['Residencial','Equipamiento','ComercialServicios','Industrial','Mixto','Complementario']
    # Filter geodataframe
    land_use = land_use[land_use['land_use'].isin(categories)]


    # Create a buffer around the street segments (e.g., 20 meters)
    zone_edges_buff = zone_edges.buffer(20, cap_style='flat')
    zone_edges_buff = gpd.GeoDataFrame(index=zone_edges_buff.index, geometry=zone_edges_buff.geometry)
    zone_edges_buff = zone_edges_buff.reset_index()

    # Check which sidewalks intersect with the edges
    intersections = gpd.overlay(zone_edges_buff, land_use, how='intersection')

    # Calculate the area of the intersections
    intersections['area_inte'] = intersections.area

    # Sum the intersection areas to get the total land use area
    area_usos_edges = intersections.groupby('osmid')['area_inte'].sum()

    # Group by land use category
    land_use_presence_edges = intersections.groupby(['osmid', 'land_use'])['area_inte'].sum()

    # Calculate the percentage of total area for each presence item (0 or 1)
    percentage_land_use_edges = land_use_presence_edges / area_usos_edges

    # Determine how many categories each osmid has
    n_land_use = intersections.groupby(['osmid'])['land_use'].unique().apply(len)

    # Create a series of all osmids present in zone_edges
    all_osmid = zone_edges.index.to_series()

    # Merge to add missing osmids
    complete_n_land_use = all_osmid.to_frame(name='id').merge(
        n_land_use.rename('n'), how='left', left_index=True, right_index=True).drop(['id'], axis=1)

    # Fill missing values with 0
    complete_n_land_use['n'] = complete_n_land_use['n'].fillna(0).astype(int)

    # Apply the entropy formula
    zone_edges['entropy'] = zone_edges.index.map(
        lambda osmid: an.calcular_entropia(osmid, complete_n_land_use['n'], percentage_land_use_edges, categories))

    zone_edges = zone_edges.rename(columns={'entropy':'land_use'})

    zone_edges = zone_edges.reset_index()
    zone_edges = zone_edges.to_crs('epsg:4326')
    zone_nodes = zone_nodes.to_crs('epsg:4326')
    zone_edges.to_file(f'../data/output/shape/land_use/{z}/{z}_land_use_edges.shp')
    zone_nodes.to_file(f'../data/output/shape/land_use/{z}/{z}_land_use_nodes.shp')

Procesando zona: aguacatala_HL
Procesando zona: moravia_LH
Procesando zona: floresta_HH
